<a href="https://colab.research.google.com/github/yourusername/url_obfuscation/blob/main/URL_Obfuscation_Single_Cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔒 URL難読化システム (シングルセル版)

このノートブックでは、URLを難読化して安全に共有・アクセスするためのシステムを提供します。下のセルを実行するだけで使用できます。

## 主な機能
- **URL難読化**: 元のURLを暗号化して隠します
- **期限付きURL**: 生成されたURLは1時間後に自動的に無効になります
- **リダイレクト対応**: リダイレクトされるURLも適切に処理します

## 使い方
1. 下のセルを実行する
2. 表示されるGUIで難読化したいURLを入力する
3. 難読化されたURLを取得・利用する

※ 初回実行時は必要なライブラリをインストールするため、少し時間がかかります

In [ ]:
# URL難読化システム - シングルセルバージョン
# 全ての機能を1つのセルに統合

# 必要なライブラリをインストール
!pip install -q requests beautifulsoup4 ipywidgets

import base64
import hashlib
import time
import urllib.parse
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import threading

# スタイルを定義
style = """
<style>
.url-box {
    margin: 10px 0;
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
    background-color: #f9f9f9;
    word-break: break-all;
}
.result-box {
    margin: 20px 0;
    padding: 15px;
    border: 1px solid #ddd;
    border-radius: 8px;
    background-color: #f5f5f5;
}
.success {
    color: #28a745;
    font-weight: bold;
}
.error {
    color: #dc3545;
    font-weight: bold;
}
.info {
    color: #17a2b8;
    font-weight: bold;
}
.tab-content {
    padding: 15px;
    border: 1px solid #ddd;
    border-top: none;
    border-radius: 0 0 5px 5px;
}
.output-area {
    max-height: 400px;
    overflow-y: auto;
    padding: 10px;
    border: 1px solid #eee;
    border-radius: 5px;
    background-color: #fafafa;
    margin-top: 10px;
}
.card {
    box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
    transition: 0.3s;
    border-radius: 5px;
    padding: 15px;
    margin: 15px 0;
    background-color: white;
}
.card:hover {
    box-shadow: 0 8px 16px 0 rgba(0,0,0,0.2);
}
.button-primary {
    background-color: #4CAF50;
    border: none;
    color: white;
    padding: 10px 24px;
    text-align: center;
    text-decoration: none;
    display: inline-block;
    font-size: 16px;
    margin: 4px 2px;
    cursor: pointer;
    border-radius: 5px;
}
.button-secondary {
    background-color: #555555;
    border: none;
    color: white;
    padding: 8px 20px;
    text-align: center;
    text-decoration: none;
    display: inline-block;
    font-size: 14px;
    margin: 4px 2px;
    cursor: pointer;
    border-radius: 5px;
}
.expiry-label {
    display: inline-block;
    padding: 5px 10px;
    background-color: #ffc107;
    color: #212529;
    border-radius: 4px;
    font-size: 0.85em;
    margin-left: 10px;
}
</style>
"""

display(HTML(style))

# 簡単な暗号化キー - 実際の使用では環境変数から取得する安全なキーを使用してください
ENCRYPTION_KEY = "proxyapi_secure_key"

def is_valid_url(url):
    """URLが有効かどうかをチェック"""
    try:
        result = urllib.parse.urlparse(url)
        # スキームとネットロックをチェック
        return all([result.scheme, result.netloc])
    except Exception as e:
        return False

def xor_encrypt(data, key):
    """単純なXOR暗号化"""
    key_bytes = key.encode('utf-8')
    if isinstance(data, str):
        data_bytes = data.encode('utf-8')
    else:
        data_bytes = data
    key_len = len(key_bytes)
    
    encrypted = bytearray()
    for i, byte in enumerate(data_bytes):
        key_byte = key_bytes[i % key_len]
        encrypted.append(byte ^ key_byte)
    
    return encrypted

def xor_decrypt(data, key):
    """XOR復号化（暗号化と同じ）"""
    if isinstance(data, str):
        data = data.encode('utf-8')
    return xor_encrypt(data, key)

def obfuscate_url(url, expiry_hours=1):
    """URL難読化（有効期限付き）"""
    try:
        url = urllib.parse.unquote(url)
        # タイムスタンプを追加
        timestamped_url = f"{url}|{datetime.now().timestamp()}"
        
        # 暗号化
        encrypted = xor_encrypt(timestamped_url, ENCRYPTION_KEY)
        encoded = base64.urlsafe_b64encode(encrypted).decode('utf-8')
        
        # チェックサム追加
        checksum = hashlib.md5(encoded.encode('utf-8')).hexdigest()[:8]
        obfuscated = f"{encoded}.{checksum}"
        
        return {
            "status": "success",
            "obfuscated_url": obfuscated,
            "original_url": url,
            "expiry": (datetime.now() + timedelta(hours=expiry_hours)).strftime("%Y-%m-%d %H:%M:%S")
        }
    except Exception as e:
        return {"status": "error", "message": f"URL難読化エラー: {str(e)}"}

def deobfuscate_url(obfuscated):
    """URL復号化（有効期限チェック付き）"""
    try:
        # 形式チェック
        parts = obfuscated.split('.')
        if len(parts) != 2:
            return {"status": "error", "message": "無効な難読化URL形式"}
        
        encoded, checksum = parts
        
        # チェックサム検証
        calculated_checksum = hashlib.md5(encoded.encode('utf-8')).hexdigest()[:8]
        if calculated_checksum != checksum:
            return {"status": "error", "message": "チェックサム検証失敗"}
        
        # Base64デコード
        try:
            encrypted = base64.urlsafe_b64decode(encoded)
        except Exception as e:
            return {"status": "error", "message": f"Base64デコードエラー: {str(e)}"}
        
        # 復号化
        decrypted = xor_encrypt(encrypted, ENCRYPTION_KEY).decode('utf-8')
        
        # タイムスタンプ抽出
        parts = decrypted.split('|')
        if len(parts) != 2:
            return {"status": "error", "message": "無効な復号化URL形式（タイムスタンプなし）"}
            
        url = parts[0]
        try:
            timestamp = float(parts[1])
            creation_time = datetime.fromtimestamp(timestamp)
            # 有効期限チェック（1時間）
            current_time = datetime.now()
            if (current_time - creation_time).total_seconds() > 3600:
                return {
                    "status": "error", 
                    "message": "URLの期限切れ（1時間以上経過）",
                    "original_url": url,
                    "created": creation_time.strftime("%Y-%m-%d %H:%M:%S"),
                    "expired": "True"
                }
            
            expiry_time = creation_time + timedelta(hours=1)
            time_left = expiry_time - current_time
            minutes_left = int(time_left.total_seconds() / 60)
            
            return {
                "status": "success",
                "original_url": url,
                "created": creation_time.strftime("%Y-%m-%d %H:%M:%S"),
                "expires": expiry_time.strftime("%Y-%m-%d %H:%M:%S"),
                "time_left": f"{minutes_left}分" if minutes_left > 0 else "まもなく期限切れ"
            }
        except Exception as e:
            return {"status": "error", "message": f"タイムスタンプ解析エラー: {str(e)}"}
    
    except Exception as e:
        return {"status": "error", "message": f"URL復号化エラー: {str(e)}"}

def proxy_request(url, wait_time=0, progress_callback=None):
    """プロキシリクエスト"""
    if not is_valid_url(url):
        return {"status": "error", "message": "無効なURLです"}
    
    try:
        if progress_callback:
            progress_callback(f"URLにリクエスト送信中...")
        
        # 指定された時間だけ待機
        if wait_time > 0:
            if progress_callback:
                progress_callback(f"{wait_time}秒待機中...")
            time.sleep(wait_time)
        
        # リクエスト送信
        resp = requests.get(
            url=url,
            headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'},
            timeout=15,
            allow_redirects=True  # リダイレクト自動追跡
        )
        
        # リダイレクトされたかチェック
        redirect_chain = None
        if url != resp.url:
            redirect_chain = [h.url for h in resp.history] + [resp.url]
        
        content_sample = resp.text[:500] + "..." if len(resp.text) > 500 else resp.text
        
        return {
            "status": "success",
            "http_status": resp.status_code,
            "content_type": resp.headers.get('content-type', ''),
            "original_url": url,
            "final_url": resp.url,
            "redirect_chain": redirect_chain,
            "content_sample": content_sample,
            "is_html": 'text/html' in resp.headers.get('content-type', '')
        }
    
    except requests.exceptions.Timeout:
        return {"status": "error", "message": "リクエストがタイムアウトしました"}
    
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"リクエストエラー: {str(e)}"}
    
    except Exception as e:
        return {"status": "error", "message": f"エラー: {str(e)}"}

# =========== GUI部分 ===========

# タブコンテンツを作成
tab = widgets.Tab()
tab_titles = ['URL難読化', 'URL復号化', 'プロキシアクセス']

# URL難読化タブ
url_input = widgets.Text(
    description='URL:',
    placeholder='https://example.com',
    layout=widgets.Layout(width='80%')
)

expiry_selector = widgets.Dropdown(
    options=[('1時間', 1), ('2時間', 2), ('6時間', 6), ('12時間', 12), ('24時間', 24)],
    value=1,
    description='有効期限:',
    disabled=False,
)

obfuscate_button = widgets.Button(
    description='URLを難読化',
    button_style='success',
    icon='lock'
)

obfuscate_output = widgets.HTML("")

obfuscate_box = widgets.VBox([
    widgets.HTML("<h3>URL難読化</h3><p>難読化したいURLを入力して「URLを難読化」ボタンをクリックしてください。</p>"),
    url_input,
    expiry_selector,
    obfuscate_button,
    obfuscate_output
])

# URL復号化タブ
deobfuscate_input = widgets.Text(
    description='難読化URL:',
    placeholder='難読化されたURLを入力',
    layout=widgets.Layout(width='80%')
)

deobfuscate_button = widgets.Button(
    description='URLを復号化',
    button_style='info',
    icon='unlock'
)

deobfuscate_output = widgets.HTML("")

deobfuscate_box = widgets.VBox([
    widgets.HTML("<h3>URL復号化</h3><p>難読化されたURLを入力して「URLを復号化」ボタンをクリックしてください。</p>"),
    deobfuscate_input,
    deobfuscate_button,
    deobfuscate_output
])

# プロキシアクセスタブ
proxy_url_input = widgets.Text(
    description='アクセスURL:',
    placeholder='https://example.com',
    layout=widgets.Layout(width='80%')
)

wait_time_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='待機時間:',
    readout=True,
    readout_format='d秒'
)

proxy_button = widgets.Button(
    description='URLにアクセス',
    button_style='primary',
    icon='globe'
)

proxy_output = widgets.HTML("")
proxy_progress = widgets.HTML("")

proxy_box = widgets.VBox([
    widgets.HTML("<h3>プロキシアクセス</h3><p>アクセスしたいURLを入力して「URLにアクセス」ボタンをクリックしてください。</p>"),
    proxy_url_input,
    wait_time_slider,
    proxy_button,
    proxy_progress,
    proxy_output
])

# タブにコンテンツを設定
tab.children = [obfuscate_box, deobfuscate_box, proxy_box]
for i, title in enumerate(tab_titles):
    tab.set_title(i, title)

# イベントハンドラ
def on_obfuscate_button_clicked(b):
    url = url_input.value.strip()
    expiry_hours = expiry_selector.value
    
    obfuscate_output.value = "<div class='info'>処理中...</div>"
    
    if not url:
        obfuscate_output.value = "<div class='error'>URLを入力してください</div>"
        return
    
    if not is_valid_url(url):
        obfuscate_output.value = "<div class='error'>有効なURLを入力してください（例: https://example.com）</div>"
        return
    
    result = obfuscate_url(url, expiry_hours)
    
    if result["status"] == "success":
        output = f"""
        <div class='result-box'>
            <div class='success'>難読化に成功しました！</div>
            <div style="margin-top:15px">
                <strong>元のURL:</strong>
                <div class="url-box">{result['original_url']}</div>
            </div>
            <div style="margin-top:15px">
                <strong>難読化されたURL:</strong>
                <div class="url-box">{result['obfuscated_url']}</div>
                <button class="button-secondary" onclick="
                    navigator.clipboard.writeText('{result['obfuscated_url']}');
                    this.innerText='コピーしました';
                    setTimeout(() => this.innerText='コピー', 2000);
                ">コピー</button>
            </div>
            <div style="margin-top:10px">
                <span class="expiry-label">期限: {result['expiry']} まで有効</span>
            </div>
        </div>
        """
        obfuscate_output.value = output
    else:
        obfuscate_output.value = f"<div class='error'>エラー: {result['message']}</div>"

def on_deobfuscate_button_clicked(b):
    obfuscated_url = deobfuscate_input.value.strip()
    
    deobfuscate_output.value = "<div class='info'>処理中...</div>"
    
    if not obfuscated_url:
        deobfuscate_output.value = "<div class='error'>難読化されたURLを入力してください</div>"
        return
    
    result = deobfuscate_url(obfuscated_url)
    
    if result["status"] == "success":
        output = f"""
        <div class='result-box'>
            <div class='success'>復号化に成功しました！</div>
            <div style="margin-top:15px">
                <strong>元のURL:</strong>
                <div class="url-box">{result['original_url']}</div>
                <button class="button-secondary" onclick="
                    navigator.clipboard.writeText('{result['original_url']}');
                    this.innerText='コピーしました';
                    setTimeout(() => this.innerText='コピー', 2000);
                ">コピー</button>
                <a href="{result['original_url']}" target="_blank" class="button-secondary">開く</a>
            </div>
            <div style="margin-top:10px">
                <span>作成日時: {result['created']}</span><br>
                <span>有効期限: {result['expires']}</span><br>
                <span class="expiry-label">残り時間: {result['time_left']}</span>
            </div>
        </div>
        """
        deobfuscate_output.value = output
    else:
        if "expired" in result and result["expired"] == "True":
            output = f"""
            <div class='result-box'>
                <div class='error'>URLの期限が切れています</div>
                <div style="margin-top:15px">
                    <strong>元のURL:</strong>
                    <div class="url-box">{result['original_url']}</div>
                </div>
                <div style="margin-top:10px">
                    <span>作成日時: {result['created']}</span><br>
                    <span class="expiry-label">1時間経過済み</span>
                </div>
            </div>
            """
            deobfuscate_output.value = output
        else:
            deobfuscate_output.value = f"<div class='error'>エラー: {result['message']}</div>"

def update_proxy_progress(message):
    proxy_progress.value = f"<div class='info'>{message}</div>"

def proxy_request_thread(url, wait_time):
    result = proxy_request(url, wait_time, update_proxy_progress)
    
    if result["status"] == "success":
        redirect_info = ""
        if result["redirect_chain"]:
            redirect_info = f"""
            <div style="margin-top:15px">
                <strong>リダイレクト:</strong>
                <ol>
            """
            for i, url in enumerate(result["redirect_chain"]):
                if i == 0:
                    redirect_info += f"<li>元のURL: {url}</li>"
                elif i == len(result["redirect_chain"]) - 1:
                    redirect_info += f"<li>最終URL: {url}</li>"
                else:
                    redirect_info += f"<li>リダイレクト {i}: {url}</li>"
            redirect_info += "</ol></div>"
        
        content_preview = ""
        if result["is_html"]:
            content_preview = f"""
            <div style="margin-top:15px">
                <strong>コンテンツプレビュー:</strong>
                <div class="output-area">
                    <pre>{result['content_sample']}</pre>
                </div>
            </div>
            <div style="margin-top:15px">
                <iframe src="{result['final_url']}" width="100%" height="300px" style="border:1px solid #ddd; border-radius:5px;"></iframe>
            </div>
            """
        
        output = f"""
        <div class='result-box'>
            <div class='success'>アクセス成功 (ステータスコード: {result['http_status']})</div>
            <div style="margin-top:15px">
                <strong>元のURL:</strong>
                <div class="url-box">{result['original_url']}</div>
            </div>
            <div style="margin-top:15px">
                <strong>最終URL:</strong>
                <div class="url-box">{result['final_url']}</div>
                <a href="{result['final_url']}" target="_blank" class="button-secondary">開く</a>
            </div>
            <div style="margin-top:10px">
                <span>コンテンツタイプ: {result['content_type']}</span>
            </div>
            {redirect_info}
            {content_preview}
        </div>
        """
        proxy_output.value = output
    else:
        proxy_output.value = f"<div class='error'>エラー: {result['message']}</div>"
    
    proxy_progress.value = ""

def on_proxy_button_clicked(b):
    url = proxy_url_input.value.strip()
    wait_time = wait_time_slider.value
    
    proxy_output.value = ""
    proxy_progress.value = "<div class='info'>処理中...</div>"
    
    if not url:
        proxy_progress.value = ""
        proxy_output.value = "<div class='error'>URLを入力してください</div>"
        return
    
    if not is_valid_url(url):
        proxy_progress.value = ""
        proxy_output.value = "<div class='error'>有効なURLを入力してください（例: https://example.com）</div>"
        return
    
    # 別スレッドでリクエスト処理を実行
    threading.Thread(target=proxy_request_thread, args=(url, wait_time)).start()

# ボタンにイベントハンドラを設定
obfuscate_button.on_click(on_obfuscate_button_clicked)
deobfuscate_button.on_click(on_deobfuscate_button_clicked)
proxy_button.on_click(on_proxy_button_clicked)

# アプリを表示
display(widgets.HTML("<h1>🔒 URL難読化システム</h1>"))
display(tab)

# 使い方ガイド
help_text = """
<div class="card">
    <h3>💡 使い方ガイド</h3>
    <p><strong>URL難読化タブ</strong>: 通常のURLを難読化して、期限付きの特殊URLに変換します</p>
    <p><strong>URL復号化タブ</strong>: 難読化されたURLを元のURLに戻し、有効期限を確認します</p>
    <p><strong>プロキシアクセスタブ</strong>: 指定したURLにアクセスして情報を取得します（リダイレクトも追跡）</p>
    <div style="margin-top:10px; padding:8px; background-color:#f8f9fa; border-radius:5px; border-left:4px solid #17a2b8">
        <p style="margin:0"><strong>ヒント:</strong> 難読化されたURLは1時間後に自動的に無効になります。長い有効期限が必要な場合は、ドロップダウンから選択してください。</p>
    </div>
</div>
"""
display(widgets.HTML(help_text))